In [1]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"


# TensorFlow ≥2.0 is required
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"



# Common imports
import numpy as np
import pandas as pd
import os

# to make this notebook's output stable across runs
np.random.seed(42)
tf.random.set_seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)


In [11]:
df_delta = pd.read_csv('df_delta.csv') # cointains labels for delta VG
df_ok = pd.read_csv('df_ok.csv')  #contains labels for ok VG

df_raw = pd.read_csv('df_raw.csv') ## contains full reading of each VG
df_raw = df_raw.set_index(['gauge_id','fillNumber'], drop=True)

df_labels = pd.concat([df_ok, df_delta], sort=False, axis=0)


df_VG = pd.merge(df_raw, df_labels, on =['gauge_id','fillNumber'])
df_VG = df_VG.set_index(['gauge_id','fillNumber'], drop=True)

## Removing categorical values
df_VG.y.replace(to_replace=['ok', 'delta'], value=[0, 1], inplace=True)

In [12]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler, MaxAbsScaler, MinMaxScaler, RobustScaler


class RowScaler(BaseEstimator, TransformerMixin):
    def __init__(self, scaling_method='Standard'):
        self.scaling_options = ['Standard', 'MinMax', 'MaxAbs', 'Robust']
        assert (scaling_method in self.scaling_options), 'scaling_method:' + scaling_method + ' not in ' + str(self.scaling_options)
        self.scaling_method = scaling_method

        return None

    def fit(self, X, y=None):
        if self.scaling_method == 'Robust':
            self.scaler = RobustScaler()
        elif self.scaling_method == 'MinMax':
            self.scaler = MinMaxScaler()
        elif self.scaling_method == 'Standard':
            self.scaler = StandardScaler()
        elif self.scaling_method == 'MaxAbs':
            self.scaler = MaxAbsScaler()
        return self
    
    def transform(self, X, y=None):
        return self.scaler.fit_transform(X.transpose()).transpose()

In [14]:
from sklearn.model_selection import train_test_split

df_unlabeled = df_raw.drop(index=df_VG.index)

X_train_ae = np.array(df_unlabeled)
X = np.array(df_VG.iloc[:, :-1])
y = np.array(df_VG.iloc[:, -1])

scaler = RowScaler()

X_train_ae = scaler.fit_transform(X_train_ae)
X = scaler.fit_transform(X)

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.33, random_state=42)

print(X_train_ae.shape)
print(X_train.shape)
print(X_valid.shape)
print(y_train.shape)
print(y_valid.shape)


(6045, 3000)
(186, 3000)
(92, 3000)
(186,)
(92,)


In [2]:
model = keras.models.load_model('./data/models/autoencoder/stacked_pca_03_dim.h5')

In [3]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential_2 (Sequential)    (None, 3)                 9012003   
_________________________________________________________________
sequential_3 (Sequential)    (None, 3000)              12012     
Total params: 9,024,015
Trainable params: 9,024,015
Non-trainable params: 0
_________________________________________________________________


In [6]:
model_version ='0001'
model_name = 'stacked_pca_ndim_003'
model_path = os.path.join(model_name, model_version)

In [9]:
tf.saved_model.save(model, model_path)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: stacked_pca_ndim_003/0001/assets


In [21]:
saved_model = tf.saved_model.load(model_path)

In [33]:
y_pred = saved_model(tf.constant(X_train[[0]], dtype=tf.float32), training=False)

In [34]:
y_pred

<tf.Tensor: shape=(1, 3000), dtype=float32, numpy=
array([[-0.77362126, -0.78488976, -0.7863362 , ..., -0.41150382,
        -0.4224767 , -0.39437792]], dtype=float32)>

In [72]:
!saved_model_cli run --dir stacked_pca_ndim_003/0001 --tag_set serve \
                     --signature_def serving_default \
                     --inputs sequential_2_input=test.npy

2020-05-04 12:45:53.195029: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2020-05-04 12:45:53.212224: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7fd2bbc22370 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2020-05-04 12:45:53.212255: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.loader.load or tf.compat.v1.saved_model.load. There will be a new function for importing SavedModels in Tensorflow 2.0.
Result for output key sequential_3:
[[-0.77362144 -0.7848898  -0.78633636 ... -0.41150385 -0.42247677
  -0.39437795]
 [-1.5438116  -1.5244832  -1.5194207  ... -0.75834846 -0.7846668
  -0.7960645 ]
 [-0.8461399  -0.8194741  -0.8274206  ... -0.7770494  -0.7

docker pull tensorflow/serving

export ML_PATH='/Users/Francesco/cernbox/SWAN_projects/VacuumGauge/vacuum_gauge'
docker run -it --rm -p 8500:8500 -p 8501:8501 \
   -v "$ML_PATH/my_mnist_model:/models/my_mnist_model" \
   -e MODEL_NAME=my_mnist_model \
   tensorflow/serving


In [70]:
X_new = X_train[:10]

In [71]:
np.save('test.npy', X_new)